# Keras starter kit [full training set, UNet]

In [1]:
# import sys

# sys.path.append('/kaggle/input/pretrainedmodels/pretrainedmodels-0.7.4')
# sys.path.append('/kaggle/input/segmentation-models-pytorch/segmentation_models.pytorch-master')
# sys.path.append('/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch-master')
# sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
# sys.path.append("/kaggle/input/einops/einops-master")

## Setup

In [2]:

import numpy as np
import torch
import torch.nn as nn
import wandb
import torchvision
import datetime
# import cupy
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning
import segmentation_models_pytorch as smp
import pytorch_lightning as pl
import pytorch_lightning.callbacks.model_checkpoint
import pytorch_lightning.plugins
from skimage.transform import resize as resize_ski
from pytorch_lightning.strategies.ddp import DDPStrategy
from pytorch_lightning.loggers import WandbLogger
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder, DecoderBlock
from timm.models.resnet import resnet10t, resnet34d, resnet50d, resnet14t, seresnext26d_32x4d
import os
from dataclasses import dataclass

from scipy.ndimage import distance_transform_edt

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import glob
import time
import PIL.Image as Image
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
from sklearn.model_selection import KFold
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data


@dataclass
class CFG():
    # Data config
    # DATA_DIR = '/kaggle/input/vesuvius-challenge-ink-detection/'
    # DATA_DIR = '/home/fummicc1/codes/competitions/kaggle-ink-detection'
    DATA_DIR = '/home/fummicc1/codes/Kaggle/kaggle-ink-detection'
    BUFFER = 160 # Half-size of papyrus patches we'll use as model inputs
    STRIDE = 80
    # Z_LIST = list(range(0, 20, 5)) + list(range(22, 34))  # Offset of slices in the z direction
    Z_LIST = list(range(22, 38, 2))
    # Z_LIST = list(range(0, 24, 8)) + list(range(24, 36, 2)) + list(range(36, 64, 10))
    Z_DIM = len(Z_LIST)  # Number of slices in the z direction. Max value is 64 - Z_START
    SHARED_HEIGHT = 4000  # Max height to resize all papyrii

    # Model config
    BATCH_SIZE = 24

    device = torch.device("cuda")
    threshold = 0.65
    num_workers = 8
    exp = 1e-7
    mask_padding = BUFFER

    num_epochs = 15
    lr = 5e-4
    eta_min_lr = 1e-6
    WANDB_NOTE = "画像サイズを大きくしてチャネルを減らした"
    pretrained = True

In [3]:
plt.imshow(Image.open(CFG.DATA_DIR + "/train/1/ir.png"), cmap="gray")

Error in callback <function flush_figures at 0x14667e6610d0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
import cupy as cp
xp = cp

delta_lookup = {
    "xx": xp.array([[1, -2, 1]], dtype=float),
    "yy": xp.array([[1], [-2], [1]], dtype=float),
    "xy": xp.array([[1, -1], [-1, 1]], dtype=float),
}

def operate_derivative(img_shape, pair):
    assert len(img_shape) == 2
    delta = delta_lookup[pair]
    fft = xp.fft.fftn(delta, img_shape)
    return fft * xp.conj(fft)

def soft_threshold(vector, threshold):
    return xp.sign(vector) * xp.maximum(xp.abs(vector) - threshold, 0)

def back_diff(input_image, dim):
    assert dim in (0, 1)
    r, n = xp.shape(input_image)
    size = xp.array((r, n))
    position = xp.zeros(2, dtype=int)
    temp1 = xp.zeros((r+1, n+1), dtype=float)
    temp2 = xp.zeros((r+1, n+1), dtype=float)
    
    temp1[position[0]:size[0], position[1]:size[1]] = input_image
    temp2[position[0]:size[0], position[1]:size[1]] = input_image
    
    size[dim] += 1
    position[dim] += 1
    temp2[position[0]:size[0], position[1]:size[1]] = input_image
    temp1 -= temp2
    size[dim] -= 1
    return temp1[0:size[0], 0:size[1]]


def forward_diff(input_image, dim):
    assert dim in (0, 1)
    r, n = xp.shape(input_image)
    size = xp.array((r, n))
    position = xp.zeros(2, dtype=int)
    temp1 = xp.zeros((r+1, n+1), dtype=float)
    temp2 = xp.zeros((r+1, n+1), dtype=float)
        
    size[dim] += 1
    position[dim] += 1

    temp1[position[0]:size[0], position[1]:size[1]] = input_image
    temp2[position[0]:size[0], position[1]:size[1]] = input_image
    
    size[dim] -= 1
    temp2[0:size[0], 0:size[1]] = input_image
    temp1 -= temp2
    size[dim] += 1
    return -temp1[position[0]:size[0], position[1]:size[1]]

def iter_deriv(input_image, b, scale, mu, dim1, dim2):
    g = back_diff(forward_diff(input_image, dim1), dim2)
    d = soft_threshold(g + b, 1 / mu)
    b = b + (g - d)
    L = scale * back_diff(forward_diff(d - b, dim2), dim1)
    return L, b

def iter_xx(*args):
    return iter_deriv(*args, dim1=1, dim2=1)

def iter_yy(*args):
    return iter_deriv(*args, dim1=0, dim2=0)

def iter_xy(*args):
    return iter_deriv(*args, dim1=0, dim2=1)

def iter_sparse(input_image, bsparse, scale, mu):
    d = soft_threshold(input_image + bsparse, 1 / mu)
    bsparse = bsparse + (input_image - d)
    Lsparse = scale * (d - bsparse)
    return Lsparse, bsparse

def denoise_image(input_image, iter_num=100, fidelity=150, sparsity_scale=10, continuity_scale=0.5, mu=1):
    image_size = xp.shape(input_image)
    #print("Initialize denoising")
    norm_array = (
        operate_derivative(image_size, "xx") + 
        operate_derivative(image_size, "yy") + 
        2 * operate_derivative(image_size, "xy")
    )
    norm_array += (fidelity / mu) + sparsity_scale ** 2
    b_arrays = {
        "xx": xp.zeros(image_size, dtype=float),
        "yy": xp.zeros(image_size, dtype=float),
        "xy": xp.zeros(image_size, dtype=float),
        "L1": xp.zeros(image_size, dtype=float),
    }
    g_update = xp.multiply(fidelity / mu, input_image)
    for i in tqdm(range(iter_num), total=iter_num):
        #print(f"Starting iteration {i+1}")
        g_update = xp.fft.fftn(g_update)
        if i == 0:
            g = xp.fft.ifftn(g_update / (fidelity / mu)).real
        else:
            g = xp.fft.ifftn(xp.divide(g_update, norm_array)).real
        g_update = xp.multiply((fidelity / mu), input_image)
        
        #print("XX update")
        L, b_arrays["xx"] = iter_xx(g, b_arrays["xx"], continuity_scale, mu)
        g_update += L
        
        #print("YY update")
        L, b_arrays["yy"] = iter_yy(g, b_arrays["yy"], continuity_scale, mu)
        g_update += L
        
        #print("XY update")
        L, b_arrays["xy"] = iter_xy(g, b_arrays["xy"], 2 * continuity_scale, mu)
        g_update += L
        
        #print("L1 update")
        L, b_arrays["L1"] = iter_sparse(g, b_arrays["L1"], sparsity_scale, mu)
        g_update += L
        
    g_update = xp.fft.fftn(g_update)
    g = xp.fft.ifftn(xp.divide(g_update, norm_array)).real
    
    g[g < 0] = 0
    g -= g.min()
    g /= g.max()
    return g

In [ ]:
def is_in_masked_zone(location, mask):
    return mask[location[0], location[1]] > 0


In [ ]:
def resize(img):
    current_height, current_width = img.shape    
    aspect_ratio = current_width / current_height
    if CFG.SHARED_HEIGHT is None:
        return img
    if CFG.SHARED_HEIGHT > current_height:
        # 解像度が既に小さい場合は行わない
        return img
    new_height = CFG.SHARED_HEIGHT
    new_width = int(CFG.SHARED_HEIGHT * aspect_ratio)
    new_size = (new_width, new_height)
    # (W, H)の順で渡すが結果は(H, W)になっている
    img = cv2.resize(img, new_size)
    return img

def load_mask(split, index): 
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/mask.png", 0) // 255    
    img = resize(img)    
    return img


def load_labels(split, index):    
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/inklabels.png", 0) // 255        
    img = resize(img)
    return img

In [ ]:
def extract_subvolume(location, volume):
    global printed
    y = location[0]
    x = location[1]
    subvolume = volume[y-CFG.BUFFER:y+CFG.BUFFER, x-CFG.BUFFER:x+CFG.BUFFER, :].astype(np.float32)
    
    return subvolume

In [ ]:
def load_volume(split, index):
    # Load the 3d x-ray scan, one slice at a time
    all = sorted(glob.glob(f"{CFG.DATA_DIR}/{split}/{index}/surface_volume/*.tif"))
    z_slices_fnames = [all[i] for i in range(len(all)) if i in CFG.Z_LIST]
    assert len(z_slices_fnames) == CFG.Z_DIM
    z_slices = []
    for z, filename in  tqdm(enumerate(z_slices_fnames)):
        img = cv2.imread(filename, -1)        
        img = resize(img)
        # img = (img / (2 ** 8)).astype(np.uint8)
        img = img.astype(np.float32) // 255
        z_slices.append(img)
    return np.stack(z_slices, axis=-1)

In [ ]:
def generate_locations_ds(volume, mask, label=None, skip_zero=False):
    is_in_mask_train = lambda x: is_in_masked_zone(x, mask)

    # Create a list to store train locations
    locations = []

    # Generate train locations
    volume_height, volume_width = volume.shape[:-1]

    for y in range(CFG.BUFFER, volume_height - CFG.BUFFER, CFG.STRIDE):
        for x in range(CFG.BUFFER, volume_width - CFG.BUFFER, CFG.STRIDE):
            if skip_zero and label is not None and np.all(label[y - CFG.STRIDE : y + CFG.STRIDE, x - CFG.STRIDE : x + CFG.STRIDE] == 0):
                print(f"skip location at (y: {y}, x: {x})")
                continue
            if is_in_mask_train((y, x)):
                locations.append((y, x))

    # Convert the list of train locations to a PyTorch tensor
    locations_ds = np.stack(locations, axis=0)
    return locations_ds

In [ ]:
tc = torch
def TTA(x:tc.Tensor,model:nn.Module):
    #x.shape=(batch,c,h,w)
    shape=x.shape
    x=[x,*[tc.rot90(x,k=i,dims=(-2,-1)) for i in range(1,4)]]
    x=tc.cat(x,dim=0)
    x, _ = model(x)
    x=x.reshape(4,shape[0], 1 ,*shape[2:])
    x=[tc.rot90(x[i],k=-i,dims=(-2,-1)) for i in range(4)]
    x=tc.stack(x,dim=0)
    return x.mean(0)

In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder

from albumentations.core.transforms_interface import ImageOnlyTransform

class SubvolumeDataset(Dataset):
    def __init__(self, locations, volume, labels, buffer, is_train: bool, return_location: bool = False):
        self.locations = locations
        self.volume = volume
        self.labels = labels        
        self.buffer = buffer
        self.is_train = is_train
        self.return_location = return_location

    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        label = None
        location = np.array(self.locations[idx])
        y, x = location[0], location[1]

        subvolume = extract_subvolume(location, self.volume)
        
        if self.labels is not None:
            label = self.labels[y - self.buffer:y + self.buffer, x - self.buffer:x + self.buffer]            
            label = np.stack([label], axis=-1)            
            
        # 段々meanは小さくなる
        mean = np.array([0.45 - i / 100 for i in range(0, CFG.Z_DIM)]).reshape(-1, 1, 1)
        # 段々stdは小さくなる
        std = np.array([0.22 - i / 300 for i in range(0, CFG.Z_DIM)]).reshape(-1, 1, 1)
        
        if self.is_train and label is not None:    
            transformed = A.Compose([
                A.HorizontalFlip(p=0.3),
                A.VerticalFlip(p=0.3),
                A.RandomScale(p=0.4, scale_limit=0.2),
                A.Transpose(p=0.4),
                A.RandomRotate90(p=0.3),
                A.ShiftScaleRotate(p=0.7, scale_limit=0.4, shift_limit=0.2),                
                A.OneOf([
                    A.GaussNoise(var_limit=[5, 20]),
                    A.GaussianBlur(blur_limit=(3, 5)),
                    A.MotionBlur(blur_limit=(3, 5)),
                ], p=0.4),
                # A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
                # A.CoarseDropout(
                #     max_holes=1, 
                #     max_width=int(CFG.BUFFER * 2 * 0.3),
                #     max_height=int(CFG.BUFFER * 2 * 0.3), 
                #     mask_fill_value=0, 
                #     p=0.5
                # ),
                A.GridDistortion(p=0.5),
                A.PixelDropout(mask_drop_value=0,  p=0.2),
                # A.CoarseDropout(
                #     max_holes=1,
                #     max_width=int(self.buffer * 0.3),
                #     max_height=int(self.buffer * 0.3),
                #     mask_fill_value=0,
                #     p=0.5,
                # ),
                # A.GridDropout(p=0.15),
                A.PadIfNeeded(min_height=self.buffer * 2, min_width=self.buffer * 2),
                A.Resize(height=self.buffer * 2, width=self.buffer * 2),
            ])(image=subvolume, mask=label)
            subvolume = transformed["image"]
            label = transformed["mask"]
            subvolume = np.transpose(subvolume, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))
            subvolume /= 255.
            subvolume = (subvolume - mean) / std       
        else:
            if label is None:
                subvolume = np.transpose(subvolume, (2, 0, 1))
                subvolume /= 255.
                subvolume = (subvolume - mean) / std
            else:
                # print("subvolume in val dataset (before aug)", subvolume, file=open("before-val-aug.log", "w")) 
                subvolume = np.transpose(subvolume, (2, 0, 1))
                label = np.transpose(label, (2, 0, 1))
                subvolume /= 255.
                subvolume = (subvolume - mean) / std
        # print("subvolume", subvolume)
        if self.return_location:
            return subvolume, location
        return subvolume, label        

In [ ]:
class SmpUnetDecoder(nn.Module):
	def __init__(
		self,
		in_channel,
		skip_channel,
		out_channel,
	):
		super().__init__()
		self.center = nn.Identity()

		i_channel = [
			in_channel,
		] + out_channel[:-1]
		s_channel = skip_channel
		o_channel = out_channel
		block = [
			DecoderBlock(i, s, o, use_batchnorm=True, attention_type=None)
			for i, s, o in zip(i_channel, s_channel, o_channel)
		]
		self.block = nn.ModuleList(block)

	def forward(self, feature, skip):
		d = self.center(feature)
		decode = []
		for i, block in enumerate(self.block):
			s = skip[i]
			d = block(d, s)
			decode.append(d)

		last = d
		return last, decode


class Net(nn.Module):
	def __init__(
		self,
	):
		super().__init__()
		self.output_type = ["inference", "loss"]

		conv_dim = 64
		encoder1_dim = [
			conv_dim,
			256,
			512,
			1024,
			2048,
		]
		decoder1_dim = [
			256,
			128,
			64,
			64,
		]

		self.encoder1 = seresnext26d_32x4d(pretrained=CFG.pretrained, in_chans=CFG.Z_DIM - 4)

		self.decoder1 = SmpUnetDecoder(
			in_channel=encoder1_dim[-1],
			skip_channel=encoder1_dim[:-1][::-1],
			out_channel=decoder1_dim,
		)
		# -- pool attention weight  
		self.weight1 = nn.ModuleList(
			[
				nn.Sequential(
					nn.Conv2d(dim, dim, kernel_size=3, padding=1),
					nn.ReLU(inplace=True),
				)
				for dim in encoder1_dim
			]
		)
		self.logit1 = nn.Conv2d(decoder1_dim[-1], 1, kernel_size=1)

		# --------------------------------
		#
		encoder2_dim = [64, 128, 256, 512]  #
		decoder2_dim = [
			128,
			64,
			32,
		]
		self.encoder2 = resnet10t(pretrained=CFG.pretrained, in_chans=decoder1_dim[-1])

		self.decoder2 = SmpUnetDecoder(
			in_channel=encoder2_dim[-1],
			skip_channel=encoder2_dim[:-1][::-1],
			out_channel=decoder2_dim,
		)
		self.logit2 = nn.Conv2d(decoder2_dim[-1], 1, kernel_size=1)

	def forward(self, batch):
		v = batch
		B, C, H, W = v.shape
		vv = [
			v[:, i : i + CFG.Z_DIM - 4]
			for i in [0, 2, 4]
		]
		K = len(vv)
		x = torch.cat(vv, 0)
		# x = v

		# ----------------------
		encoder = []
		e = self.encoder1
		x = e.conv1(x)
		x = e.bn1(x)
		x = e.act1(x)
		encoder.append(x)
		x = F.avg_pool2d(x, kernel_size=2, stride=2)
		x = e.layer1(x)
		encoder.append(x)
		x = e.layer2(x)
		encoder.append(x)
		x = e.layer3(x)
		encoder.append(x)
		x = e.layer4(x)
		encoder.append(x)
		# print('encoder', [f.shape for f in encoder])

		for i in range(len(encoder)):
			e = encoder[i]
			f = self.weight1[i](e)
			_, c, h, w = e.shape
			f = rearrange(f, "(K B) c h w -> B K c h w", K=K, B=B, h=h, w=w)  #
			e = rearrange(e, "(K B) c h w -> B K c h w", K=K, B=B, h=h, w=w)  #
			w = F.softmax(f, 1)
			e = (w * e).sum(1)
			encoder[i] = e

		feature = encoder[-1]
		skip = encoder[:-1][::-1]
		last, decoder = self.decoder1(feature, skip)
		logit1 = self.logit1(last)
  
		logit1 = F.interpolate(
			logit1, size=(H, W), mode="bilinear", align_corners=False, antialias=True
		)

		# ----------------------
		x = last  # .detach()
		# x = F.avg_pool2d(x,kernel_size=2,stride=2)
		encoder = []
		e = self.encoder2
		x = e.layer1(x)
		encoder.append(x)
		x = e.layer2(x)
		encoder.append(x)
		x = e.layer3(x)
		encoder.append(x)
		x = e.layer4(x)
		encoder.append(x)

		feature = encoder[-1]
		skip = encoder[:-1][::-1]
		last, decoder = self.decoder2(feature, skip)
		logit2 = self.logit2(last)
		logit2 = F.interpolate(
			logit2, size=(H, W), mode="bilinear", align_corners=False, antialias=True
		)
		return logit1, logit2


In [ ]:
# ref - https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/discussion/397288
def fbeta_score(preds, targets, threshold, beta=0.5, smooth=1e-5):
    preds_t = torch.where(preds > threshold, 1.0, 0.0).float()
    y_true_count = targets.sum()
    
    ctp = preds_t[targets==1].sum()
    cfp = preds_t[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

In [ ]:
def dice_coef_torch(prob_preds, targets, beta=0.5, smooth=1e-5):
    # No need to binarize the predictions
    # prob_preds = torch.sigmoid(preds)

    # flatten label and prediction tensors
    prob_preds = prob_preds.view(-1).float()
    targets = targets.view(-1).float()

    intersection = (prob_preds * targets).sum()

    dice = ((1 + beta**2) * intersection + smooth) / ((beta**2) * prob_preds.sum() + targets.sum() + smooth)

    return dice



class Model(pl.LightningModule):
    training_step_outputs = []
    validation_step_outputs = []
    test_step_outputs = [[], []]

    def __init__(self, **kwargs):
        super().__init__()

        self.model = Net()        

        # self.loss1 = nn.BCEWithLogitsLoss(
        #     pos_weight=torch.tensor([0.7])
        # )
        # self.loss2 = nn.BCEWithLogitsLoss(
        #     pos_weight=torch.tensor([0.7])
        # )
        self.loss1 = smp.losses.TverskyLoss(
            smp.losses.BINARY_MODE,
            log_loss=False,
            from_logits=True, 
            smooth=1e-6,
            alpha=0.6,
            beta=0.4,
        )
        self.loss2 = smp.losses.TverskyLoss(
            smp.losses.BINARY_MODE,
            log_loss=False,
            from_logits=True, 
            smooth=1e-6,
            alpha=0.6,
            beta=0.4,
        )

    def forward(self, image, stage):
        mask = TTA(image, self.model)
        return mask

    def shared_step(self, batch, stage):
        subvolumes, labels = batch

        image, labels = subvolumes.float(), labels.float()        
        assert image.ndim == 4
        
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0
        
        # print("labels", labels.max(), labels.min())

        assert labels.max() <= 1.0 and labels.min() >= 0

        logit1, logit2 = self.forward(image, stage)
        
        # print("segmentation_out", segmentation_out.shape)

        loss = 0.3 * self.loss1(logit1, labels) + 0.7 * self.loss2(logit2, labels)
        
        prob2 = torch.sigmoid(logit2)

        pred_mask = (prob2 > CFG.threshold).float()
        
        # print("pred_mask", pred_mask)
        
        score = fbeta_score(pred_mask, labels, threshold=CFG.threshold)

        tp, fp, fn, tn = smp.metrics.get_stats(
            pred_mask.long(), labels.long(), mode="binary"
        )

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
            "score": score,
        }

    def shared_epoch_end(self, outputs, stage):
        # aggregate step metics
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])
        loss = torch.mean(torch.Tensor([x["loss"] for x in outputs]))
        fbeta_score = torch.mean(torch.Tensor([x["score"] for x in outputs]))

        per_image_iou = smp.metrics.iou_score(
            tp, fp, fn, tn, reduction="micro-imagewise"
        )

        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")

        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
            f"{stage}_loss": loss.item(),
            f"{stage}_tp": tp.sum().int().item(),
            f"{stage}_fp": fp.sum().int().item(),
            f"{stage}_fn": fn.sum().int().item(),
            f"{stage}_tn": tn.sum().int().item(),
            f"{stage}_score": fbeta_score.item(),
        }

        self.log_dict(metrics, prog_bar=True, sync_dist=True)

    def training_step(self, batch, batch_idx):
        out = self.shared_step(batch, "train")
        self.training_step_outputs.append(out)
        return out

    def on_train_epoch_end(self):
        out = self.shared_epoch_end(self.training_step_outputs, "train")
        self.training_step_outputs.clear()
        return out

    def validation_step(self, batch, batch_idx):
        out = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(out)
        return out

    def on_validation_epoch_end(self):
        out = self.shared_epoch_end(self.validation_step_outputs, "valid")
        self.validation_step_outputs.clear()
        return out

    def test_step(self, batch, batch_idx):
        global predictions_map, predictions_map_counts

        patch_batch, loc_batch = batch

        loc_batch = loc_batch.long()
        patch_batch = patch_batch.float()
        predictions = self.forward(patch_batch, "test")
        predictions = predictions.sigmoid()
 
        predictions = torch.permute(predictions, (0, 2, 3, 1)).squeeze(dim=-1)
        predictions = (
            predictions.cpu().numpy()
        )
        loc_batch = loc_batch.cpu().numpy()
        
        self.test_step_outputs[0].extend(loc_batch)
        self.test_step_outputs[1].extend(predictions)        
        return loc_batch, predictions


    def on_test_epoch_end(self):
        global predictions_map, predictions_map_counts

        locs = np.array(self.test_step_outputs[0])
        preds = np.array(self.test_step_outputs[1])
        print("locs", locs.shape)
        print("preds", preds.shape)

        new_predictions_map = np.zeros_like(predictions_map[:, :, 0])
        new_predictions_map_counts = np.zeros_like(predictions_map_counts[:, :, 0])

        for (y, x), pred in zip(locs, preds):
            new_predictions_map[
                y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER
            ] += pred
            new_predictions_map_counts[
                y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER
            ] += 1
        
        new_predictions_map /= new_predictions_map_counts + CFG.exp
        new_predictions_map = new_predictions_map[:, :, np.newaxis]
        predictions_map = np.concatenate(
            [predictions_map, new_predictions_map], axis=-1
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=CFG.lr)
        
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.8, patience=2,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "valid_loss"},
        }


In [ ]:
class EnsembleModel:
    def __init__(self, test_loader, test_volume):
        super().__init__()
        self.test_loader = test_loader
        self.test_volume = test_volume
        self.list = []
        for fold in [1, 2, 3, 4]:
            _model = Model.load_from_checkpoint(
                f"weights/weights_fold-{fold}.ckpt",               
            )
            trainer = pl.Trainer(
                accelerator="gpu",
                devices="1",
                enable_checkpointing=False,
            )

            self.list.append((_model, trainer))
    
    def forward(self):
        global predictions_map, predictions_map_counts
        predictions_map = np.empty_like(self.test_volume[:, :, 0])[:, :, np.newaxis].astype(np.float64)
        predictions_map_counts = np.empty_like(predictions_map).astype(np.uint8)
        for i, (model, trainer) in enumerate(self.list):
            model: Model = model
            model.test_step_outputs = [[], []]
            model.eval()
            trainer.test(
                model=model,
                dataloaders=self.test_loader,
                verbose=True,
            )
            self.list[i] = None
            gc.collect()
        predictions_map = predictions_map[:, :, 1:].mean(axis=-1, keepdims=True)

In [ ]:


import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
from skimage.transform import resize as resize_ski
import pathlib
import gc

def compute_predictions_map(split, index):
    global predictions_map
    global predictions_map_counts
    
    print(f"Load data for {split}/{index}")

    test_volume = load_volume(split=split, index=index)
    test_mask = load_mask(split=split, index=index)    
    
    #get coord
    stride = CFG.BUFFER
    H,W,D  = test_volume.shape

    ##pad #assume H,W >size
    px, py = W % stride, H % stride
    if (px != 0) or (py != 0):
        px = stride - px
        py = stride - py
        test_volume = np.pad(test_volume, [(0, py), (0, px), (0, 0)], constant_values=0)
        test_mask = np.pad(test_mask, [(0, py), (0, px)], constant_values=0)  
        
    test_locations = generate_locations_ds(test_volume, test_mask)  
    
    print(f"{len(test_locations)} test locations (before filtering by mask)")

    # filter locations inside the mask
    test_locations = [loc for loc in test_locations if is_in_masked_zone(loc, test_mask)]
    
    print(f"{len(test_locations)} test locations (after filtering by mask)")

    test_ds = SubvolumeDataset(test_locations, test_volume, None, CFG.BUFFER, is_train=False, return_location=True)
    test_loader = DataLoader(test_ds, batch_size=CFG.BATCH_SIZE, num_workers=CFG.num_workers, pin_memory=True)        

    # # shape: (Y, X, C)
    predictions_map = np.zeros_like(test_volume[:, :, 0])[:, :, np.newaxis].astype(np.float64)
    predictions_map_counts = np.zeros_like(predictions_map).astype(np.uint8)

    print("test_volume.shape", test_volume.shape)
    print("predictions_map.shape", predictions_map.shape)

    print(f"Compute predictions")
    
    model = EnsembleModel(test_loader, test_volume)
    model.forward()
    del model
    del test_locations
    del test_loader
    del test_ds
    del test_volume
    del test_mask
    gc.collect()
    
    # print("predictions_map", predictions_map, file=open("predictions_map", "w"))
    return predictions_map[:H, :W, :]


In [ ]:
# Fast run length encoding, from https://www.kaggle.com/code/hackerpoet/even-faster-run-length-encoder/script
# (H, W)の形式
def rle (img: np.ndarray, threshold: float):
    flat_img = img.flatten()
    flat_img = np.where(flat_img > threshold, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix

    return " ".join(map(str, sum(zip(starts_ix, lengths), ())))

In [ ]:
def update_submission(predictions_map, index):
    rle_ = rle(predictions_map, threshold=CFG.threshold)
    print(f"{index}," + rle_, file=open('submission.csv', 'a'))
    # print(f"{index}," + rle_, file=open('/kaggle/working/submission.csv', 'a'))

In [ ]:
predictions_map = None
predictions_map_counts = None
print("Id,Predicted", file=open('submission.csv', 'w'))
kind = "test"
folder = pathlib.Path(CFG.DATA_DIR) / kind
for p in list(folder.iterdir()):
    index = p.stem
    predictions_map = compute_predictions_map(split=kind, index=index)
    original_size = cv2.imread(CFG.DATA_DIR + f"/{kind}/{index}/mask.png", 0).shape[:2]
    resized_predictions_map = resize_ski(predictions_map, (original_size[0], original_size[1], 1), anti_aliasing=True).squeeze(axis=-1)
    print("original size", original_size)
    resized_predictions_map=xp.array(resized_predictions_map)
    resized_predictions_map=denoise_image(resized_predictions_map, iter_num=250)
    resized_predictions_map=resized_predictions_map.get()
    update_submission(resized_predictions_map, index)
    resized_predictions_map = np.where(resized_predictions_map >= CFG.threshold, 255, 0)
    plt.imsave(f"{index}_{str(CFG.threshold)}_{str(CFG.BUFFER)}.png", resized_predictions_map, cmap="gray")

Load data for test/a


8it [00:05,  1.60it/s]


1648 test locations (before filtering by mask)
1648 test locations (after filtering by mask)
test_volume.shape (2880, 6400, 8)
predictions_map.shape (2880, 6400, 1)
Compute predictions


/home/fummicc1/anaconda3/envs/ink-detection-3_8/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/fummicc1/anaconda3/envs/ink-detection-3_8/lib/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fummicc1/anaconda3/envs/ink-detection-3_8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `

Testing: 0it [00:00, ?it/s]

...

locs (1486, 2)
preds (1486, 320, 320)


You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing: 0it [00:00, ?it/s]

locs (1486, 2)
preds (1486, 320, 320)
original size (5454, 6330)


100%|██████████| 250/250 [01:11<00:00,  3.48it/s]
